In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import json
tfk  = tf .keras
tfkl = tfk.layers

from utils import *
from guided_ig import *
from tqdm import tqdm
import pickle

from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing

In [2]:
p_xml = '/media/big/imagenet/data/ILSVRC/Annotations/CLS-LOC/val'
p_jpg = '/media/big/imagenet/data/ILSVRC/Data/CLS-LOC/val'
files_xml = sorted(os.listdir(p_xml))
files_jpg = sorted(os.listdir(p_jpg))

In [3]:
with open('imagenet_class_index.json', 'r') as fp:
    label_json = json.loads(fp.read())
labels = np.array([
    [k, *v]
    for k, v in label_json.items()
])

In [4]:
model = tf.keras.applications.InceptionV3()
def grad_func(x, idx):
    x = tf.constant(x)[tf.newaxis]
    with tf.GradientTape() as tape:
        tape.watch(x)
        pred = model(x)
        pred = pred[:, idx]
    return np.squeeze(tape.gradient(pred, x).numpy())

2021-12-05 18:14:33.469926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 18:14:33.475414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 18:14:33.475693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 18:14:33.476568: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
p_save = '/media/big/imagenet/computed-gig'

for f_xml, f_jpg in tqdm(zip(files_xml, files_jpg)): 
    with open(f'{p_xml}/{f_xml}') as fp:
        xml = fp.read()

    path = f'{p_jpg}/{f_jpg}'
    img = io_ops.read_file(path)
    img = image_ops.decode_image(
        img, 
        channels = 3, 
        expand_animations = False
    )
    img = image_ops.resize_images_v2(
        img, 
        (299, 299), 
        method = image_preprocessing.get_interpolation('bilinear')
    )
    img = preprocess_iv3(img)[0]
    p5  = model(img[tf.newaxis]).numpy().flatten().argsort()[::-1][:5]
    y   = xml.split('\n')[13][8:-7]

    check_match = np.argwhere(labels[p5, 1] == y)
    if len(check_match) > 0:
        check_idx = p5[check_match][0, 0]
        
        # Guided IG
        gig = np.stack([
            unbounded_guided_ig(
                img.numpy(),
                baseline, 
                200, 
                lambda x: grad_func(x, check_idx), 
                0.1
            )
            for baseline in [
                np.zeros_like(img) - 1, # To the model, black is -1
                np.ones_like (img)      # White is 1
            ]
        ])
        
        # Anchored Guided IG (GIG(20))
        gig20 = np.stack([
            anchored_guided_ig(
                img.numpy(),
                baseline, 
                lambda x: grad_func(x, check_idx), 
                200, 
                0.1,
                anchors = 20
            )
            for baseline in [
                np.zeros_like(img) - 1,
                np.ones_like (img)
            ]
        ])
        res = np.concatenate([gig, gig20])
        
        with open(f'{p_save}/{f_xml}-{f_jpg}.pickle', 'wb') as fp:
            pickle.dump({
                'res': res,
                'p5' : p5,
                'y'  : y
            }, fp)

0it [00:00, ?it/s]2021-12-05 18:14:35.456162: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2021-12-05 18:14:35.631876: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
31it [40:36, 83.67s/it]